<a href="https://colab.research.google.com/github/choccoba/hbv-tahoe-analysis/blob/main/Step0_fix_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔧 GSE182159 배치 파일 재통합

**목적**: 저장된 배치 파일들을 올바르게 통합하여 GSE182159_processed.h5ad 재생성

| 배치 | 세포 수 |
|------|--------|
| batch_0 | 31,233 |
| batch_1 | 36,988 |
| batch_2 | 44,922 |
| batch_3 | 80,837 |
| batch_4 | 49,020 |
| **총합** | **243,000** |

In [1]:
# @title STEP 1: 환경 설정
from google.colab import drive
drive.mount('/content/drive')

!pip install anndata scanpy pandas -q

import anndata as ad
import pandas as pd
import os

RAW_PATH = '/content/drive/MyDrive/tahoe-x1/viral_hepatitis/raw_data'
print(f'✅ 환경 설정 완료!')
print(f'📁 경로: {RAW_PATH}')

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.2/174.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.1/284.1 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 93.9 MB/s eta 0:00:00
✅ 환경 설정 완료!
📁 경로: /content/drive/MyDrive/tahoe-x1/viral_hepatitis/raw_data


In [2]:
# @title STEP 2: 배치 파일 확인
batch_files = sorted([f for f in os.listdir(RAW_PATH) if f.startswith('GSE182159_batch_')])
print(f'📦 발견된 배치 파일: {len(batch_files)}개\n')

total_cells = 0
for bf in batch_files:
    adata_temp = ad.read_h5ad(f'{RAW_PATH}/{bf}', backed='r')
    print(f'   {bf}: {adata_temp.n_obs:,} cells, {adata_temp.n_vars:,} genes')
    total_cells += adata_temp.n_obs

print(f'\n📊 총 세포 수: {total_cells:,}')

📦 발견된 배치 파일: 5개

   GSE182159_batch_0.h5ad: 31,233 cells, 24,452 genes
   GSE182159_batch_1.h5ad: 36,988 cells, 24,452 genes
   GSE182159_batch_2.h5ad: 44,922 cells, 24,452 genes
   GSE182159_batch_3.h5ad: 80,837 cells, 24,452 genes
   GSE182159_batch_4.h5ad: 49,020 cells, 24,452 genes

📊 총 세포 수: 243,000


In [3]:
# @title STEP 3: 배치 파일 통합
import gc

print('🔗 배치 파일 통합 중...')
print('   (이 작업은 몇 분이 소요될 수 있습니다)\n')

adata_list = []
for i, bf in enumerate(batch_files):
    print(f'   Loading {bf}...')
    adata_batch = ad.read_h5ad(f'{RAW_PATH}/{bf}')
    adata_list.append(adata_batch)
    print(f'   ✅ {bf}: {adata_batch.n_obs:,} cells loaded')
    gc.collect()

print(f'\n🔗 Concatenating {len(adata_list)} batches...')
adata = ad.concat(adata_list, join='outer')
adata.var_names_make_unique()

print(f'\n✅ 통합 완료!')
print(f'   세포 수: {adata.n_obs:,}')
print(f'   유전자 수: {adata.n_vars:,}')
print(f'   샘플 수: {adata.obs["sample"].nunique()}')

# 메모리 정리
del adata_list
gc.collect()

🔗 배치 파일 통합 중...
   (이 작업은 몇 분이 소요될 수 있습니다)

   Loading GSE182159_batch_0.h5ad...
   ✅ GSE182159_batch_0.h5ad: 31,233 cells loaded
   Loading GSE182159_batch_1.h5ad...
   ✅ GSE182159_batch_1.h5ad: 36,988 cells loaded
   Loading GSE182159_batch_2.h5ad...
   ✅ GSE182159_batch_2.h5ad: 44,922 cells loaded
   Loading GSE182159_batch_3.h5ad...
   ✅ GSE182159_batch_3.h5ad: 80,837 cells loaded
   Loading GSE182159_batch_4.h5ad...
   ✅ GSE182159_batch_4.h5ad: 49,020 cells loaded

🔗 Concatenating 5 batches...

✅ 통합 완료!
   세포 수: 243,000
   유전자 수: 24,452
   샘플 수: 46


0

In [5]:
# @title STEP 4: 메타데이터 매핑 (HBV 상태 추가)
metadata_file = f'{RAW_PATH}/sample_metadata.csv'

if os.path.exists(metadata_file):
    print('📋 메타데이터 로드 중...')
    metadata_df = pd.read_csv(metadata_file)
    print(f'   샘플 수: {len(metadata_df)}')
    print(f'   컬럼: {metadata_df.columns.tolist()}')

    # GSM ID 추출 (샘플명에서)
    # sample 형식: GSM5519467_P190604_Blood_1
    adata.obs['gsm_id'] = adata.obs['sample'].str.extract(r'(GSM\d+)')[0]

    # 매핑 생성
    meta_dict = metadata_df.set_index('sample_id').to_dict()

    # Stage 매핑 (HBV 상태)
    if 'Stage' in meta_dict:
        adata.obs['hbv_stage'] = adata.obs['gsm_id'].map(meta_dict['Stage'])
        print(f'\n📊 HBV Stage 분포:')
        print(adata.obs['hbv_stage'].value_counts())

    # donor 매핑
    if 'donor' in meta_dict:
        adata.obs['donor'] = adata.obs['gsm_id'].map(meta_dict['donor'])
        print(f'\n📊 Donor 수: {adata.obs["donor"].nunique()}')

    print('\n✅ 메타데이터 매핑 완료!')
else:
    print(f'⚠️ 메타데이터 파일이 없습니다: {metadata_file}')

📋 메타데이터 로드 중...
   샘플 수: 46
   컬럼: ['sample_id', 'title', 'source', 'tissue', 'Stage', 'donor']

📊 HBV Stage 분포:
hbv_stage
IA    62545
IT    49179
AR    45452
AC    43245
NL    42579
Name: count, dtype: int64

📊 Donor 수: 23

✅ 메타데이터 매핑 완료!


In [6]:
# @title STEP 5: 통합 파일 저장
output_file = f'{RAW_PATH}/GSE182159_processed.h5ad'

print(f'💾 저장 중: {output_file}')
print('   (이 작업은 몇 분이 소요될 수 있습니다)\n')

adata.write_h5ad(output_file)

print('=' * 60)
print('🎉 GSE182159 데이터 통합 및 저장 완료!')
print('=' * 60)
print(f'\n📁 저장된 파일: {output_file}')
print(f'\n📊 데이터 요약:')
print(f'   세포 수: {adata.n_obs:,}')
print(f'   유전자 수: {adata.n_vars:,}')
print(f'   샘플 수: {adata.obs["sample"].nunique()}')
print(f'\n📋 메타데이터 컬럼:')
print(f'   {list(adata.obs.columns)}')
print('\n✅ 다음 단계: Step1_HBV_real.ipynb 실행')

💾 저장 중: /content/drive/MyDrive/tahoe-x1/viral_hepatitis/raw_data/GSE182159_processed.h5ad
   (이 작업은 몇 분이 소요될 수 있습니다)

🎉 GSE182159 데이터 통합 및 저장 완료!

📁 저장된 파일: /content/drive/MyDrive/tahoe-x1/viral_hepatitis/raw_data/GSE182159_processed.h5ad

📊 데이터 요약:
   세포 수: 243,000
   유전자 수: 24,452
   샘플 수: 46

📋 메타데이터 컬럼:
   ['sample', 'tissue', 'gsm_id', 'hbv_stage', 'donor']

✅ 다음 단계: Step1_HBV_real.ipynb 실행


In [7]:
# @title (선택) 저장된 파일 검증
print('🔍 저장된 파일 검증 중...')
adata_check = ad.read_h5ad(output_file, backed='r')
print(f'   Shape: {adata_check.shape}')
print(f'   ✅ 검증 완료! 세포 수: {adata_check.n_obs:,}')

🔍 저장된 파일 검증 중...
   Shape: (243000, 24452)
   ✅ 검증 완료! 세포 수: 243,000
